In [5]:
from subprocess import check_output
from os.path import expanduser
import os

In [6]:
home = expanduser("~")

SCRIPT = 'shore-deconvolve.py '
DATA_DIR = os.path.join(home, 'Devel/test/samuel/in4michi')
OUT_DIR = os.path.join(home, 'Devel/test/samuel/in4michi/test_results')
OPTIONS = ['hpsd', 'nonneg', 'none']
venv = '. ' + os.path.join(home, 'Devel/venvs/bonndit/bin/activate') + ' && '

for option in OPTIONS:
    command = venv + SCRIPT + '{di}/data.nii.gz {out}/response.npz \
{out}/odf{opt}.nrrd --pos {opt} --csf {out}/csf{opt}.nrrd \
--gm {out}/gm{opt}.nrrd --wm {out}/wm{opt}.nrrd'.format(di=DATA_DIR, opt=option, out=OUT_DIR)
    out = check_output(command, shell = True)
    print(out)

b'order (radial / angular): 4 / 4\n[[281.77762371   0.           0.         ...   0.         455.33339525\n  651.25522943]\n [281.77762371   0.           0.         ...   0.         455.33339525\n  651.25522943]\n [281.77762371   0.           0.         ...   0.         455.33339525\n  651.25522943]\n ...\n [ 93.2258914    0.           0.         ...   0.          82.49415443\n   29.13357179]\n [ 93.2258914    0.           0.         ...   0.          82.49415443\n   29.13357179]\n [ 93.2258914    0.           0.         ...   0.          82.49415443\n   29.13357179]]\nCondition number of M: inf\nOptimizing...\n  0.0%\x08\x08\x08\x08\x08\x08\x08\x08   done   \n'
b'order (radial / angular): 4 / 4\n[[281.77762371   0.           0.         ...   0.         455.33339525\n  651.25522943]\n [281.77762371   0.           0.         ...   0.         455.33339525\n  651.25522943]\n [281.77762371   0.           0.         ...   0.         455.33339525\n  651.25522943]\n ...\n [ 93.2258914    0.  

In [4]:
import pyshtools as sh

In [34]:
%%time
sh.legendre.PlmIndex(6,4)

CPU times: user 9 µs, sys: 2 µs, total: 11 µs
Wall time: 15 µs


25

In [10]:
def _get_size(a, r):
    if (a % 2) == 1 or (r % 2) == 1:
        return 0
    return sum([l * r - l * l + r // 2 + l // 2 * 3 + 1 for l in range(0, a + 1, 2)])
MAX_ORDER = 12
SIZES = [[_get_size(a, r) if r >= a else 0 for a in range(MAX_ORDER + 1)] for r in range(MAX_ORDER + 1)]

In [71]:
%%time
sh.legendre.PlON(15, 0.4)

CPU times: user 22 µs, sys: 0 ns, total: 22 µs
Wall time: 28.1 µs


array([ 0.28209479,  0.195441  , -0.16400361, -0.32839517, -0.09563013,
        0.25321148,  0.29764219, -0.01594072, -0.31054865, -0.23210796,
        0.12518587,  0.33220361,  0.14032553, -0.22006898, -0.31628272,
       -0.03279847])

In [56]:
from bonndit.michi.esh import legendre_slow

In [74]:
%%time
legendre(2, 0.6)

CPU times: user 28 µs, sys: 5 µs, total: 33 µs
Wall time: 39.3 µs


array([ 0.28209479,  0.17416376, -0.50914892,  0.32912265, -0.50914892,
        0.17416376])